## TPRU: Tax Model Demonstration
#### August 23, 2019

The [Policy Simulation Library](https://www.pslmodels.org) (PSL) hosts open-source, policy-related models that conform to certain standards for model I/O, documentation, and testing.

In this notebook, we illustrate how three of these models ([`Tax-Calculator`](https://github.com/PSLmodels/Tax-Calculator), [`Behavioral-Responses`](https://github.com/PSLmodels/Behavioral-Responses), [`Cost-of-Capital-Calculator`](https://github.com/PSLmodels/Cost-of-Capital-Calculator), [`OG-USA`](https://github.com/PSLmodels/OG-USA)) can work together to provide a broad analysis of set of tax policies.


#### Setup
Before running this notebook, you must open a terminal window and install the Tax-Calculator (taxcalc), Cost-of-Capital-Calculator (ccc), and OG-USA (ogusa) packages.
* [`Tax-Calculator`](https://github.com/PSLmodels/Tax-Calculator) (taxcalc): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels taxcalc`
* [`Behavioral-Responses`](https://github.com/PSLmodels/Behavioral-Responses) (behresp): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels behresp`
* [`Cost-of-Capital-Calculator`](https://github.com/PSLmodels/Cost-of-Capital-Calculator) (ccc): This package can be downloaded from Anaconda. Go to terminal and type `conda install -c PSLmodels ccc`.
* [`OG-USA`](https://github.com/PSLmodels/OG-USA) (ogusa): This package must be cloned and set up a specific environment. Follow the instructions in the main [`README.md`](https://github.com/PSLmodels/OG-USA/blob/master/README.md).

In [ ]:
# imports
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
# import PSL models
import taxcalc
import ccc
import behresp
from ogusa import output_plots as op
# to print bokeh plots inline
output_notebook()
# to print matplotlib plots inline
%matplotlib inline

## Revenue estimates and distributional analysis of the personal income tax with Tax-Calculator

We'll start by using the `Tax-Calculator` model, a microsimulation model of individual income taxes and payroll taxes.  This model can be used for revenue estimation, distributional analysis, effective tax rate calculations, and other uses.

As an example to work with, we'll simulate the Tax Cuts and Jobs Act.  Since this represents current law, which is the baseline policy in Tax-Calculator, we'll need to specify changes to this that represent tax law in 2017:

In [ ]:
law2017_url = ('https://raw.githubusercontent.com/'
              'PSLmodels/Tax-Calculator/master/taxcalc/'
              'reforms/2017_law.json')
law2017_dict = taxcalc.Policy.read_json_reform(law2017_url)
law2017_dict

In [ ]:
# Create calculators for 2017 law and TCJA
pol = taxcalc.policy.Policy()
rec = taxcalc.records.Records.cps_constructor()
tc_tcja = taxcalc.calculator.Calculator(policy=pol, records=rec)
pol.implement_reform(law2017_dict)
tc_2017 = taxcalc.calculator.Calculator(policy=pol, records=rec)

### Revenue Estimation

In [ ]:
# Compute annual revenue for each year in a 10 year budget window
# Do this for 2017 law and the TCJA
rev_2017 = {'iit': {}, 'payroll': {}}
rev_tcja = {'iit': {}, 'payroll': {}}
for t in range(2018, 2028):
    tc_2017.advance_to_year(t)
    tc_2017.calc_all()
    tc_tcja.advance_to_year(t)
    tc_tcja.calc_all()
    rev_2017['iit'][t] = tc_2017.weighted_total('iitax')
    rev_2017['payroll'][t] = tc_2017.weighted_total('payrolltax')
    rev_tcja['iit'][t] = tc_tcja.weighted_total('iitax')
    rev_tcja['payroll'][t] = tc_tcja.weighted_total('payrolltax')

In [ ]:
# make table of revenue estimates
rev_2017_df = pd.DataFrame.from_dict(rev_2017).T
rev_tcja_df = pd.DataFrame.from_dict(rev_tcja).T
diff_df = (rev_tcja_df - rev_2017_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_df.sum(axis=1)

### Distributional Analysis and Effective Rates Calculations

In [ ]:
# Restart calculation to get back to first year
pol = taxcalc.policy.Policy()
tc_tcja = taxcalc.calculator.Calculator(policy=pol, records=rec)
pol.implement_reform(law2017_dict)
tc_2017 = taxcalc.calculator.Calculator(policy=pol, records=rec)
tc_2017.advance_to_year(2018)
tc_tcja.advance_to_year(2018)
tc_2017.calc_all()
tc_tcja.calc_all()
# Average tax rate plot
atr_plot = tc_2017.atr_graph(tc_tcja)
show(atr_plot)

In [ ]:
pch_plot = tc_2017.pch_graph(tc_tcja)
show(pch_plot)

In [ ]:
# Distributional table
tc_2017.difference_table(tc_tcja, 'standard_income_bins', 'combined')

In [ ]:
# Marginal tax rate plot
mtr_plot = tc_2017.mtr_graph(tc_tcja)
show(mtr_plot)

## Including behavioral responses

Using the `Behavior-Response` package, we can apply partial-equilibrium behavioral responses to the analysis using `Tax-Calculator`:

In [ ]:
# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.6}
# apply a behavioral response to the change in after-tax income from the reform
_, df2br = behresp.response(tc_2017, tc_tcja, response_elasticities)

# Calculate reform income tax liabilities for 2018 with behavioral response
itax_rev2br = (df2br['iitax'] * df2br['s006']).sum()
# Calculate baseline income tax liabilities for 2018
base_rev = tc_2017.weighted_total('iitax')
# Calculate reform income tax liabilities for 2018 without behavior responses
reform_rev_static = tc_tcja.weighted_total('iitax')

# Change in total income tax revenue estimates for 2018 (policy-baseline)
# (estimates in billons of dollars)
print('{}_STATIC_itax_rev_change($B)= {:.3f}'.format(
    2018, (reform_rev_static - base_rev) * 1e-9))
print('{}_DYNAMIC_itax_rev_change($B)= {:.3f}'.format(
    2018, (itax_rev2br - base_rev) * 1e-9))

## Effective tax rates with the Cost-of-Capital-Calculator

Another important consideration in tax policy are the incentive effects of taxation on business investment.  The Cost-of-Capital-Calculator (CCC) is a model that allows one to explore effects of tax policy on the cost of capital and effective tax rates (ETR).

CCC computes the the cost of capital CCC and ETRS across business entity tax treatment (e.g., corporate vs non-corporate), indusry, and asset type.  When considering the effects of the individual income tax on savings and investment decisions, CCC uses the output from Tax-Calculator.

Here, we use CCC to provide an analysis of the effects of the TCJA on investment incentives.

In [ ]:
# Load asset data
assets = ccc.data.Assets()
assets.df.head(n=5)

In [ ]:
# To compare two policies, create two model specifications
# Then use the Calculator class to compute ETRs (and more)
# across the two specifications
# Create an instance of the Specification class
spec_2017 = ccc.parameters.Specification(year=2018, call_tc=True, iit_reform=law2017_dict)
business_law_2017 = business_tax_reform = {
    'CIT_rate': 0.35, 'BonusDeprec_3yr': 0.50, 'BonusDeprec_5yr': 0.50,
    'BonusDeprec_7yr': 0.50, 'BonusDeprec_10yr': 0.50,
    'BonusDeprec_15yr': 0.50, 'BonusDeprec_20yr': 0.50}
spec_2017.update_specification(business_law_2017)
calc_2017 = ccc.calculator.Calculator(spec_2017, assets)
spec_tcja = ccc.parameters.Specification(year=2018, call_tc=True, iit_reform={})
calc_tcja = ccc.calculator.Calculator(spec_tcja, assets)

In [ ]:
# Summary table
pd.options.display.float_format = '{:.2f}%'.format
calc_2017.summary_table(
    calc_tcja, output_variable='metr',
    include_land=True, include_inventories=True)

In [ ]:
# Plot to show variation in METRs across assets
rplot = calc_2017.range_plot(
    calc_tcja, corporate=True, output_variable='metr',
    include_land=False, include_inventories=False)
show(rplot)

In [ ]:
# Visualizing changes by asset
aplot = calc_2017.grouped_bar(calc_tcja, include_land=True, include_inventories=True)
show(aplot)

In [ ]:
# Visualizing changes by industry
iplot = calc_2017.grouped_bar(calc_tcja, group_by_asset=False)
show(iplot)

In [ ]:
bplot = calc_2017.asset_bubble(calc_tcja)
show(bplot)

## Macro results with OG-USA

OG-USA is a overlapping generations model used to evaluate fiscal policy.  OG-USA is calibrated to the U.S. economy, but we've run this source code with a calibration to Italy (Calling the model OG-ITA) and are in the process of calibrating OG-India.

OG-USA will show aggregate effects of tax policy (as well as impacts across generations and skill groups) from the current period until the economy reaches its long-run steady state.

OG-USA takes some time to run (1+ hours to solve the model), so we've cached some results where we simulated the transition from 2017 law to the TCJA.  

In [ ]:
# Load in OG-USA results and parameterizations
from ogusa import utils
base_tpi = utils.safe_read_pickle('./TCJA_Results/OUTPUT_BASELINE/partial/TPI/TPI_vars.pkl')
base_ss = utils.safe_read_pickle('./TCJA_Results/OUTPUT_BASELINE/partial/SS/SS_vars.pkl')
base_params = utils.safe_read_pickle('./TCJA_Results/OUTPUT_BASELINE/partial/model_params.pkl')
reform_tpi = utils.safe_read_pickle('./TCJA_Results/OUTPUT_REFORM/partial/TPI/TPI_vars.pkl')
reform_ss = utils.safe_read_pickle('./TCJA_Results/OUTPUT_REFORM/partial/SS/SS_vars.pkl')
reform_params = utils.safe_read_pickle('./TCJA_Results/OUTPUT_REFORM/partial/model_params.pkl')

In [ ]:
# Plot OG-USA output; interest rates
rplot = op.plot_aggregates(
    base_tpi, base_params, reform_tpi=reform_tpi,
    reform_params=reform_params, var_list=['r_gov'],
    plot_type='levels', num_years_to_plot=22,
    vertical_line_years=[
        base_params.start_year + base_params.tG1,
        base_params.start_year + base_params.tG2],
    plot_title='Real Interest Rates Under Baseline and Reform')

In [ ]:
# Plot OG-USA output; government debt
dplot = op.plot_gdp_ratio(base_tpi, base_params, reform_tpi, reform_params,
                   var_list=['D'], num_years_to_plot=100,
                   start_year=2019, vertical_line_years=[
                           base_params.start_year + base_params.tG1,
                           base_params.start_year + base_params.tG2],
                   plot_title='Debt-to-GDP')

In [ ]:
# Plot OG-USA output; percentage changes in macro vars (Y, K, L, C)
pct_chg_plot = op.plot_aggregates(base_tpi, base_params, reform_tpi=reform_tpi,
                    reform_params=reform_params,
                    var_list=['Y', 'K', 'L', 'C'], plot_type='pct_diff',
                    num_years_to_plot=100,
                    vertical_line_years=[
                        base_params.start_year + base_params.tG1,
                        base_params.start_year + base_params.tG2],
                    plot_title='Percentage Changes in Macro Aggregates')

In [ ]:
# Plot OG-USA output; Levels of GDP
pct_chg_plot = op.plot_aggregates(base_tpi, base_params, reform_tpi=reform_tpi,
                    reform_params=reform_params,
                    var_list=['Y'], plot_type='cbo',
                    num_years_to_plot=22,
                    vertical_line_years=[
                        base_params.start_year + base_params.tG1,
                        base_params.start_year + base_params.tG2],
                    plot_title='GDP Levels under Baseline and Reform')

In [ ]:
# Plot OG-USA output; percentage changes in investment
pct_chg_plot = op.plot_aggregates(base_tpi, base_params, reform_tpi=reform_tpi,
                    reform_params=reform_params,
                    var_list=['I_total'], plot_type='pct_diff',
                    num_years_to_plot=22,
                    vertical_line_years=[
                        base_params.start_year + base_params.tG1,
                        base_params.start_year + base_params.tG2],
                    plot_title='Percentage Changes in Investment')

In [ ]:
# Plot OG-USA output; percentage changes in macro vars (Y, K, L, C)
pct_chg_plot = op.plot_aggregates(base_tpi, base_params, reform_tpi=reform_tpi,
                    reform_params=reform_params,
                    var_list=['G', 'total_revenue'], plot_type='levels',
                    num_years_to_plot=22,
                    vertical_line_years=[
                        base_params.start_year + base_params.tG1,
                        base_params.start_year + base_params.tG2],
                    plot_title='Fiscal Vars')

In [ ]:
op.ss_profiles(base_ss, base_params, reform_ss=None,
                reform_params=None, by_j=False, var='nssmat',
                plot_data=True,
                plot_title=None, path=None)